In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/runi-nlp-2023/nlp_2023_test.csv
/kaggle/input/runi-nlp-2023/nlp_2023_train.csv
/kaggle/input/runi-nlp-2023/nlp_2023_sample.csv


In [2]:
!pip install -q --upgrade accelerate einops xformers

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchdata 0.6.0 requires torch==2.0.0, but you have torch 2.0.1 which is incompatible.


In [3]:
from transformers import TrainingArguments,Trainer

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [4]:
df = pd.read_csv('/kaggle/input/runi-nlp-2023/nlp_2023_train.csv')
df

,Unnamed: 0,text1,text2,Similarity
0,0,How many customers does Apple have?,How many servers does apple have?,0.0
1,1,Pregnancy k stating month m kya Khana chaiye?,Do those who want heaven most do the most for ...,0.0
2,2,What changes are there in MHRD scholarship giv...,On what criteria are the scholarships given to...,0.0
3,3,What are some best book for Computer Architect...,Which writer should I follow for computer arch...,0.0
4,4,What are some unexpected things first-time vis...,What are some unexpected things first-time vis...,0.0
...,...,...,...,...
69995,69995,How can I earn rupees 1 lakh within a month?,How can I earn 1 lakh in a day?,0.0
69996,69996,Why would you hate the movie 'PINK'?,What do you not like about the movie 'Pink'?,1.0
69997,69997,How can Bitcoin be hacked?,How is the theft of Bitcoin possible by hacking?,1.0
69998,69998,She asked to be excused from last week 's Cabi...,She took the highly unusual step of skipping c...,1.0


In [5]:
df.describe(include='object')

,text1,text2
count,70000,70000
unique,60659,61937
top,A man is playing a guitar.,A man is playing a guitar.
freq,13,18


In [6]:
df['input'] = 'TEXT1: ' + df.text1 + '; TEXT2: ' + df.text2

In [7]:
df.input.head()

0    TEXT1: How many customers does Apple have?; TE...
1    TEXT1: Pregnancy k stating month m kya Khana c...
2    TEXT1: What changes are there in MHRD scholars...
3    TEXT1: What are some best book for Computer Ar...
4    TEXT1: What are some unexpected things first-t...
Name: input, dtype: object

In [8]:
from datasets import Dataset, DatasetDict
ds = Dataset.from_pandas(df)

In [9]:
ds

Dataset({
    features: ['Unnamed: 0', 'text1', 'text2', 'Similarity', 'input'],
    num_rows: 70000
})

In [10]:
model_nm = 'microsoft/deberta-v3-small'

from transformers import AutoModelForSequenceClassification,AutoTokenizer
tokz = AutoTokenizer.from_pretrained(model_nm)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
def tok(x):
    return tokz(x['input'])

In [12]:
tok_ds = ds.map(tok, batched=True)

  0%|          | 0/70 [00:00<?, ?ba/s]

In [13]:
row = tok_ds[0]
row['input'], row['input_ids']

('TEXT1: How many customers does Apple have?; TEXT2: How many servers does apple have?',
 [1,
  54453,
  435,
  294,
  577,
  386,
  887,
  490,
  2013,
  286,
  302,
  346,
  54453,
  445,
  294,
  577,
  386,
  5239,
  490,
  6038,
  286,
  302,
  2])

In [14]:
tokz.vocab['▁for']

270

In [15]:
tok_ds = tok_ds.rename_columns({'Similarity':'labels'}) # for transformers 

In [16]:
dds = tok_ds.train_test_split(0.2, seed = 0)
dds

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text1', 'text2', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 56000
    })
    test: Dataset({
        features: ['Unnamed: 0', 'text1', 'text2', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 14000
    })
})

In [18]:
eval_df =pd.read_csv('/kaggle/input/runi-nlp-2023/nlp_2023_test.csv')
eval_df.describe()

,Unnamed: 0
count,3000.000000
mean,1499.500000
std,866.169729
min,0.000000
25%,749.750000
50%,1499.500000
75%,2249.250000
max,2999.000000


In [42]:
eval_df.head()

,Unnamed: 0,text1,text2,input
0,0,her husband got tickets to the Giants game for...,did your husband get tickets to the giants gam...,TEXT1: her husband got tickets to the Giants g...
1,1,the fundraiser is now at the middle school,the fundraiser is now at the middle school,TEXT1: the fundraiser is now at the middle sch...
2,2,Is helicopter parenting in the United States i...,Is helicopter parenting going away anytime soon?,TEXT1: Is helicopter parenting in the United S...
3,3,A man is buttering a piece of bread.,A slow loris hanging on a cord.,TEXT1: A man is buttering a piece of bread.; T...
4,4,"Asked if the delegates could leave on Friday ,...","Asked if the delegates could leave on Friday ,...",TEXT1: Asked if the delegates could leave on F...


In [19]:
dds = tok_ds.train_test_split(0.2, seed = 0)
dds

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text1', 'text2', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 56000
    })
    test: Dataset({
        features: ['Unnamed: 0', 'text1', 'text2', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 14000
    })
})

In [20]:
eval_df['input'] = 'TEXT1: ' + eval_df.text1 + '; TEXT2: ' + eval_df.text2 
eval_ds = Dataset.from_pandas(eval_df).map(tok, batched=True)

  0%|          | 0/3 [00:00<?, ?ba/s]

In [21]:
def corr(x,y): 
    return np.corrcoef(x,y)[0][1]

In [22]:
def corr_d(preds):
    return {'pearson': corr(*preds)}

In [23]:
# pip install accelerate -U

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.


In [24]:
# pip install transformers[torch]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.


In [25]:
bs = 32
epochs = 4
lr = 8e-5

In [26]:
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')

In [27]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                  tokenizer=tokz, compute_metrics=corr_d)

Some weights of the model checkpoint at microsoft/deberta-v3-small were not used when initializing DebertaV2ForSequenceClassification: ['mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.classifier.bias', 'mask_predictions.LayerNorm.weight', 'mask_predictions.dense.weight', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.classifier.weight']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from 

In [28]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Pearson
1,0.142300,0.097353,0.763927
2,0.079500,0.092970,0.794738
3,0.039900,0.094576,0.793338
4,0.021300,0.095437,0.788459


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

TrainOutput(global_step=3500, training_loss=0.06739526312691825, metrics={'train_runtime': 1802.7743, 'train_samples_per_second': 124.253, 'train_steps_per_second': 1.941, 'total_flos': 5108580185911680.0, 'train_loss': 0.06739526312691825, 'epoch': 4.0})

In [60]:
preds = trainer.predict(eval_ds).predictions.astype(float)
preds[0]

array([0.31186035])

In [64]:
preds = np.clip(preds, 0, 1)

In [67]:
preds[34]

array([0.71878314])

In [33]:
len(preds)

3000

In [71]:
for i in range(len(preds)):
    if preds[i] >= 0.6:
        preds[i] = 1
    else:
        preds[i] = 0

In [72]:
preds = list(map(int, preds))

In [73]:
import datasets

submission = datasets.Dataset.from_dict({
    'Id': eval_ds['Unnamed: 0'],
    'Category': preds
})

submission.to_csv('submit.csv', index=False)

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

19902

In [ ]:
# Change model and try -->